#**Trabalho de Mineração de dados**


**Alunos:** Halliday Gauss e Guilherme Augusto



# **O Problema:** Spaceship Titanic

No ano de 2912, foi recebido uma transmissão de quatro anos-luz de distância.

A nave espacial Titanic com quase 13.000 passageiros a bordo, partiu em sua viagem inaugural transportando emigrantes do nosso sistema solar para três exoplanetas recém-habitáveis ​​orbitando estrelas próximas.

Durante o percurso a nave colidiu com uma anomalia do espaço-tempo escondida dentro de uma nuvem de poeira. Embora a nave tenha permanecido intacta, quase metade dos passageiros foi transportada para uma dimensão alternativa!

Para ajudar as equipes de resgate e recuperar os passageiros perdidos, você é desafiado a prever quais passageiros foram transportados pela anomalia usando registros recuperados do sistema de computador danificado da nave espacial.



# **Motivação:** 

Deseja-se ajudar a salvar os seres humanos e mudar a história !

Após uma análise dessa base de dados tirada do Kaggle é possível perceber dados inconsistentes, irrelavantes e nulos. Portanto, é uma boa oportunidade para treinar os conhecimentos de Mineração de Dados e uma posterior Classificação (predição) através de algoritmos de Inteligência Artificial.

# Objetivo:

O objetivo do trabalho é realizar a mineração dos dados na base de dados, de maneira que os mesmos se tornem úteis para que a aplicação de um algoritmo de classificação seja eficiente e aceitável.

Através de um conjunto de teste deve-se realizar a predição dos dados, visando saber se os passageiros foram parar em outra dimensão ou não.

#Descrição dos Atributos
------------------------------------------------------------------------

**PassengerId** 

 - Um ID exclusivo para cada passageiro.

 - Atributo Categórico Ordinal

------------------------------------------------------------------------

**HomePlanet** 
- O planeta de onde o passageiro partiu, normalmente seu planeta de residência permanente.
- Atributo Categórico Nominal

------------------------------------------------------------------------
**CryoSleep** 
- Indica se o passageiro optou por ser colocado em animação suspensa durante a viagem.
- Atributo Binário Simétrico

------------------------------------------------------------------------

**Cabin** 
- O número da cabine onde o passageiro está hospedado.
- Atributo Categórico Ordinal

------------------------------------------------------------------------

**Destination** 
- O planeta para o qual o passageiro irá desembarcar.
- Atributo Categórico Nominal
------------------------------------------------------------------------

**Age** 
- A idade do passageiro.
- Atributo Discreto

------------------------------------------------------------------------

**VIP** 
- Se o passageiro pagou pelo serviço VIP especial durante a viagem.
- Atributo Binário Simétrico

------------------------------------------------------------------------

**RoomService** 
- Valor que o passageiro cobrou pelo serviço de quarto.
- Atributo Contínuo

------------------------------------------------------------------------

**FoodCourt** 
- Valor que o passageiro gastou na praça de alimentação.
- Atributo Contínuo

------------------------------------------------------------------------

**ShoppingMall** 
- Valor que o passageiro gastou no shopping.
- Atributo Contínuo

------------------------------------------------------------------------

**Spa** 
- Valor que o passageiro gastou no spa.
- Atributo Contínuo

------------------------------------------------------------------------
**VRDeck** 
- Valor que o passageiro gastou no deck de RV.
- Atributo Contínuo

------------------------------------------------------------------------

**Name** 
- O nome do passageiro.
- Atributo Categórico Nominal
------------------------------------------------------------------------

**Transported** 
- Se o passageiro foi transportado para outra dimensão.
- Atributo Binário Simétrico (O que deve ser predito)
------------------------------------------------------------------------

#Importações

In [ ]:
# Bibliotecas processamento
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

#Biblioteca seaborn
import seaborn as sns

from sklearn.impute import SimpleImputer # Fit e Transform em dados nulos
from sklearn.preprocessing import OneHotEncoder # Fazer one hot encoder
from sklearn.preprocessing import LabelEncoder #Label Enconder substitui as classes por números.
from sklearn import preprocessing # Para fazer a normalização dos dados


from mlxtend.frequent_patterns import apriori # Biblioteca para encontrar transações frequentes
from mlxtend.frequent_patterns import association_rules # Biblioteca para gerar as regras de associação
from mlxtend.preprocessing import TransactionEncoder # Para fazer o One Hot

# Ler base de dados

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
# Tamanho da base de Dados
tamanho_do_df = len(df.index)
print("Tamanho da base de dados:", tamanho_do_df, "\n\n")

print("Algumas instâncias da base de dados:")
df.head()

# Funções Auxiliares

Função que vai gerar um gráfico de pizza dos atributos nominais.

In [ ]:
def plotar_grafico_de_pizza_nominal(df, nome_do_atributo, titulo = None):

  rotulos = list(df[nome_do_atributo].unique())

  data = [df[nome_do_atributo].eq(rot).sum() for rot in rotulos]

  if df[nome_do_atributo].isnull().sum() > 0:
    data[-1] = df[nome_do_atributo].isnull().sum() 
  
  if titulo is None:
    titulo = "Distribuição do atributo - " + nome_do_atributo

  explode = [0.1*i for i in range(len(rotulos))]

  wp = { 'linewidth' : 1, 'edgecolor' : "green" } 
  def func(pct, allvalues): 
    absolute = int(pct / 100.*np.sum(allvalues)) 
    return "{:.1f}%\n({:d} g)".format(pct, absolute) 
  fig, ax = plt.subplots(figsize =(10, 7)) 
  wedges, texts, autotexts = ax.pie(data,  
                                  autopct = lambda pct: func(pct, data), 
                                  explode = explode,  
                                  labels = rotulos, 
                                  shadow = True,  
                                  startangle = 90, 
                                  wedgeprops = wp, 
                                  textprops = dict(color ="magenta")) 
  ax.legend(wedges, rotulos, 
          loc ="center left", 
          bbox_to_anchor =(1, 0, 0.5, 1)) 
  
  plt.setp(autotexts, size = 8, weight ="bold") 
  ax.set_title(titulo) 
  plt.show() 
  plt.show() 

Função que vai plotar o gráfico de pizza da quantidade de atributos nulos e não nulos.

In [ ]:
def plotar_grafico_de_pizza_nulos(df, nome_do_atributo):

  print("\nGráfico de Pizza - Valores Nulos - " + nome_do_atributo)
  titulo = "\nGráfico de Pizza - Valores Nulos - " + nome_do_atributo
  nulos = ['Nulos', 'Não Nulos'] 
  data = [df[nome_do_atributo].isnull().sum(), df[nome_do_atributo].count()] 

  explode = (0.1, 0.0) 
  colors = ("beige", "orange") 
  wp = { 'linewidth' : 1, 'edgecolor' : "green" } 
  def func(pct, allvalues): 
    absolute = int(pct / 100.*np.sum(allvalues)) 
    return "{:.1f}%\n({:d} g)".format(pct, absolute) 
  fig, ax = plt.subplots(figsize =(10, 7)) 
  wedges, texts, autotexts = ax.pie(data,  
                                  autopct = lambda pct: func(pct, data), 
                                  explode = explode,  
                                  labels = nulos, 
                                  shadow = True, 
                                  colors = colors, 
                                  startangle = 90, 
                                  wedgeprops = wp, 
                                  textprops = dict(color ="magenta")) 
  ax.legend(wedges, nulos, 
          loc ="center left", 
          bbox_to_anchor =(1, 0, 0.5, 1)) 
  
  plt.setp(autotexts, size = 8, weight ="bold") 
  ax.set_title(titulo) 
  plt.show() 
  plt.show() 

Função que vai fazer a análise estatística do atributo.

In [ ]:
def analise_estatistica_atributo(df, nome_do_atributo): 
  # Analisar Atributo - Age
  print("Análise Estatística do atributo -", nome_do_atributo)
  print("Valor médio:", round(df[nome_do_atributo].mean(),2))
  print("Valor máximo:",round(df[nome_do_atributo].max(), 2))
  print("Valor mínimo:", round(df[nome_do_atributo].min(), 2))
  print("Valor desvio padrão:", round(df[nome_do_atributo].std(), 2))
  print("25% do valores contidos até:", round(df[nome_do_atributo].describe()[4], 2))
  print("50% do valores contidos até:", round(df[nome_do_atributo].describe()[5], 2))
  print("75% do valores contidos até:", round(df[nome_do_atributo].describe()[6], 2))

Função que vai plotar o histograma de um atributo.

In [ ]:
def plotar_histograma_atributo(df, nome_do_atributo):
  print("Histograma")

  plt.title("Histograma - " + nome_do_atributo, fontsize=20)
  plt.xlabel(nome_do_atributo)
  plt.ylabel('Quantidade')
  plt.hist(df[nome_do_atributo], rwidth=0.9)

  plt.show()

Função que vai plotar o gráfico de dispersão de um atributo.

In [ ]:
def plotar_grafico_de_dispersao_atributo(df, nome_do_atributo):
  print("\nGráfico de Dispersão")
  plt.scatter(df[nome_do_atributo], range(len(df[nome_do_atributo])))
  plt.xlabel(nome_do_atributo)
  plt.ylabel('Instância')
  plt.title('Gráfico de Dispersão - ' + nome_do_atributo)
  plt.show()

Função que vai plotar relação do atributo com o rótulo através de um bloxplot.

In [ ]:
def plotar_relacao_atributo_rotulo_bloxplot(df, nome_do_atributo):
  print("\nRelação do atributo " + nome_do_atributo + " com o Rótulo")
  df.boxplot(by='Transported', column = nome_do_atributo)


Função que vai plotar relação do atributo com o rótulo através de um gráfico de barras.

In [ ]:
def plotar_relacao_atributo_rotulo_barras(df, nome_do_atributo):
  print("\nRelação do atributo " + nome_do_atributo + " com o Rótulo")

  transported = df[df.Transported == True]
  not_transported = df[df.Transported == False]

  transported = transported[nome_do_atributo]
  not_transported = not_transported[nome_do_atributo]

  plt.figure(figsize=(8, 6))
  plt.title('Distribuição do atributo ' + nome_do_atributo)
  plt.xlabel(nome_do_atributo)
  plt.ylabel('Quantidade de pessoas')
  plt.hist(transported,
          alpha=0.5, label='Transportado', color='#FF26E1')
  plt.hist(not_transported,
          alpha=0.5, label='Não Transportado', color='#2DB200')
  plt.legend(loc='upper right')

Função que vai analisar a unicidade de cada valor do atributo.

In [ ]:
def unicidade(df, nome_do_atributo):
  print("Unicidade -", nome_do_atributo)
  print("Quantidades únicas:" , len(df[nome_do_atributo].unique()))

Função que vai plotar a relação de cada possível valor do atributo nominal com o rótulo.

In [ ]:
def plotar_relacao_atributo_rotulo_pizza(df, nome_do_atributo):
  print("Relação do atributo", nome_do_atributo, " com o Rótulo")

  rotulos = list(df[nome_do_atributo].unique())

  for rotulo in rotulos:
    plotar_grafico_de_pizza_nominal(df[df[nome_do_atributo].eq(rotulo)], "Transported", "Gráfico de Distribuição do atributo - " +
                                    nome_do_atributo + " - " + str(rotulo) + " em relação ao rótulo Transported")
    print("\n")

# Análise dos Atributos Numéricos

Analisar -> Idade - Age

In [ ]:
analise_estatistica_atributo(df, "Age")

In [ ]:
plotar_histograma_atributo(df, "Age")

Existem mais pessoas entre 20 e 40 anos.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "Age")

Em idade existem 178 valores nulos.

In [ ]:
plotar_grafico_de_dispersao_atributo(df, "Age")

Em idade não existem valores aberrantes e inconsistentes.

In [ ]:
plotar_relacao_atributo_rotulo_bloxplot(df, "Age")

In [ ]:
plotar_relacao_atributo_rotulo_barras(df, "Age")

Apesar dos dados estarem bem distribuídos pela idade em relação ao rótulo.
É possível perceber que:
  - A maioria das pessoas de 0 a 15 anos, de 40 a 45 anos, e de 55 a 70 anos, foram transportadas para outra dimensão.
  - A maioria das pessoas de 15 a 40 anos, de 45 a 55 anos, e de 70 a 80 anos, não foram transportadas para outra dimensão.

Analisar -> Gastos com Serviço de Quarto - RoomService

In [ ]:
# Analisar Serviço de Quarto - RoomService
analise_estatistica_atributo(df, "RoomService")

In [ ]:
plotar_histograma_atributo(df, "RoomService")

Os gastos com serviço de quarto estão mais concentrados em valores menores que 2000.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "RoomService")

Existem 180 valores nulos.

In [ ]:
plotar_grafico_de_dispersao_atributo(df, "RoomService")

Valores Aberrantes -> acima de 8000

Não possui valores Inconsistentes !

In [ ]:
plotar_relacao_atributo_rotulo_bloxplot(df, "RoomService")

In [ ]:
plotar_relacao_atributo_rotulo_barras(df, "RoomService")

É possível perceber que quem gastou mais com o serviço de quarto não foi transportado para outra dimensão.

Analisar Gastos com Praça de Alimentação - FoodCourt

In [ ]:
analise_estatistica_atributo(df, "FoodCourt")

In [ ]:
plotar_histograma_atributo(df, "FoodCourt")

Os gastos com alimentação estão mais concentrados em valores menores que 5000.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "FoodCourt")

Possui 182 valores nulos.

In [ ]:
plotar_grafico_de_dispersao_atributo(df, "FoodCourt")

Valores aberrantes -> acima de 20000

Não possui valores Inconsistentes.

In [ ]:
plotar_relacao_atributo_rotulo_bloxplot(df, "FoodCourt")

In [ ]:
plotar_relacao_atributo_rotulo_barras(df, "FoodCourt")

É possível perceber que quanto mais se gastou com na praça de alimentação maior a chance da pessoa ser transportada para outra dimensão.

Analisar Gastos com Shopping - ShoppingMall

In [ ]:
analise_estatistica_atributo(df, "ShoppingMall")

In [ ]:
plotar_histograma_atributo(df, "ShoppingMall")

Percebe-se que os gastos com shopping são, em maioria, menores que 5000.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "ShoppingMall")

Possui 207 valores nulos.

In [ ]:
plotar_grafico_de_dispersao_atributo(df, "ShoppingMall")

Valores Aberrantes -> acima de 5000.

Não possui valores Inconsistentes.

In [ ]:
plotar_relacao_atributo_rotulo_bloxplot(df, "ShoppingMall")

In [ ]:
plotar_relacao_atributo_rotulo_barras(df, "ShoppingMall")

Percebe-se que quantos mais se gasta no shopping, maior a chance da pessoa ser transportada para outra dimensão.

Analisar Gastos com Spa - Spa

In [ ]:
analise_estatistica_atributo(df, "Spa")

In [ ]:
plotar_histograma_atributo(df, "Spa")

Os gastos com Spa estão mais concentrados em valores menores que 5000.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "Spa")

Possui 182 valores nulos.

In [ ]:
plotar_grafico_de_dispersao_atributo(df, "Spa")

Valores Aberrantes -> acima de 15000.

Não possui valores Inconsistentes.

In [ ]:
plotar_relacao_atributo_rotulo_bloxplot(df, "Spa")

In [ ]:
plotar_relacao_atributo_rotulo_barras(df, "Spa")

É possível perceber que quanto mais a pessoa gastou no Spa, menor a chance dela ser transportada para outra dimensão.

Analisar Gastos com VRDeck - VRDeck

In [ ]:
analise_estatistica_atributo(df, "VRDeck")

In [ ]:
plotar_histograma_atributo(df, "VRDeck")

Os valores gastos com VRDeck estão mais concentrados em valores menores que 5000.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "VRDeck")

Possui 187 valores nulos.

In [ ]:
plotar_grafico_de_dispersao_atributo(df, "VRDeck")

Valores Aberrantes -> acima de 15000.

Não possui valores Inconsistentes.

In [ ]:
plotar_relacao_atributo_rotulo_bloxplot(df, "VRDeck")

In [ ]:
plotar_relacao_atributo_rotulo_barras(df, "VRDeck")

Percebe-se que quanto mais o passageiro gastou com VRDeck, menor a chance dele ser transportado para outra dimensão.

Conclui-se que:
  - Pessoas localizadas quartos mais caros, no spa e no vrdeck, tem uma chance menor de serem transportadas para outra dimensão
  - Pessoas localizadas na praça de alimentação e shopping, tem maiores chances de serem transportadas para outra dimensão.

#Análise dos Atributos Categóricos (binários, ordinais e nominais)

Analisar Id do Passageiro - PassengerId

In [ ]:
unicidade(df, "PassengerId")

In [ ]:
plotar_grafico_de_pizza_nulos(df, "PassengerId")

Não possui valores nulos.

Não possui valores Aberrantes.

Não possui valores Inconsistentes.

Analisar Planeta Natal - HomePlanet

In [ ]:
plotar_grafico_de_pizza_nominal(df, "HomePlanet")

Mais pessoas partiram da Terra.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "HomePlanet")

Possui 200 valores nulos.

Não possui Valores Aberrantes.

Não possui Valores Inconsistentes.

In [ ]:
plotar_relacao_atributo_rotulo_pizza(df, "HomePlanet")

É possível perceber que pessoas que partiram do planeta:
 - Europa e Marte: tem mais chance de serem transportadas para outra dimensão.
 - Terra: tem menos chance de isso ocorrer.

Analisar Cabine de Sono - CryoSleep

In [ ]:
plotar_grafico_de_pizza_nominal(df, "CryoSleep")

Mais pessoas optaram por não entrar na cabine do sono.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "CryoSleep")

Possui 216 valores nulos.

Não possui Valores Aberrantes !

Não possui Valores Inconsistentes !

In [ ]:
plotar_relacao_atributo_rotulo_pizza(df, "CryoSleep")

É possível perceber que:
 - Pessoas que foram para a Cabine do Sono tem mais chance de serem transportadas para outra dimensão, dos que as que não foram.

Analisar Cabine - Cabin

In [ ]:
unicidade(df, "Cabin")


In [ ]:
plotar_grafico_de_pizza_nulos(df, "Cabin")

Possui 198 valores nulos.

Não possui Valores Aberrantes !

Não possui Valores Inconsistentes !

In [ ]:
#plotar_relacao_atributo_rotulo_pizza(df, "Cabin")

Analisar Destino - Destination

In [ ]:
plotar_grafico_de_pizza_nominal(df, "Destination")

Percebe-se que mais pessoas estão se destinando ao planeta TRAPPIST-1e.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "Destination")

Possui 181 valores nulos.

Não possui Valores Aberrantes !

Não possui Valores Inconsistentes !

In [ ]:
plotar_relacao_atributo_rotulo_pizza(df, "Destination")

É possível perceber que pessoas que estavam se destinando ao planeta:
 - TRAPPIST-1e: tem menos chance de serem transportadas para outra dimensão.
 - 55 Cancri e, PSO J318.5-22: tem mais chance de isso ocorrer.

Analisar Passageiro Vip - VIP

In [ ]:
plotar_grafico_de_pizza_nominal(df, "VIP")

Percebe-se que apenas uma pequena parcela dos passageiros eram VIPs.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "VIP")

Possui 202 valores nulos.

Não possui Valores Aberrantes !

Não possui Valores Inconsistentes !

In [ ]:
plotar_relacao_atributo_rotulo_pizza(df, "VIP")

Percebe-se que a maioria dos passageiros VIP's' não foram transportados para outra dimensão. No entanto, isso permanece bem distribuído entre os passageiros não VIP's'.

Analisar Nome do Passageiro - Name

In [ ]:
unicidade(df, "Name")

In [ ]:
plotar_grafico_de_pizza_nulos(df, "Name")

Possui 199 valores nulos.

Não possui Valores Aberrantes !

Não possui Valores Inconsistentes !

In [ ]:
# Possui Nomes Repetidos
print(Counter(df.Name))

Analisar Transportado - Transported

In [ ]:
plotar_grafico_de_pizza_nominal(df, "Transported")

Percebe-se que está bem distribuído a quantidade de passageiros que foram transportados para outra dimensão e os que não foram.

In [ ]:
plotar_grafico_de_pizza_nulos(df, "Transported")

Não possui valores nulos.

Não possui Valores Aberrantes !

Não possui Valores Inconsistentes !

#Prévia Correlação entre os atributos

In [ ]:
# análise de correlação

correlation = df.corr()

# plot da matriz de correlação

plot = sns.heatmap(correlation, annot = True, fmt=".1f", linewidths=.6)
plot

A partir do Mapa de Calor de Correlação é possível perceber que os atributos  possuem baixíssima correlação.

Uma melhor análise da correlação dos atributos pode ser feita após uma discretização dos dados e a realização da transformação dos atributos nominais em númericos juntamente do One Hot Encoding.

Essa correlação vai melhorar principalmente quando o rótulo for transformado em inteiro.

#Limpeza e transformação de dados

# Preencher Dados Nulos

In [ ]:
df.count()

In [ ]:
# Preencher Dados Nulos com Média

atributos = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]

imputers_continuos = dict()
# Usar media
for atributo in atributos: 
  imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
  imputer = imputer.fit(df[[atributo]])
  imputers_continuos[atributo] = imputer
  df[atributo] = imputers_continuos[atributo].transform(df[[atributo]])


In [ ]:
# Usar Desconhecido

atributos = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Name"]

imputers_nominais = SimpleImputer(strategy='constant', fill_value="Desconhecido", missing_values=np.nan)

for atributo in atributos: 
  imputers_nominais = imputers_nominais.fit(df[[atributo]])
  df[atributo] = imputers_nominais.transform(df[[atributo]])


In [ ]:
df.count()

Para os atributos:
 - Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck, foi utilizada a média para preencher os valores nulos.
 - HomePlanet, CryoSleep, Cabin, Destination, VIP, Name, foi utilizada a substituição pela constante "Desconhecido", para prencher os valores nulos.

#Suavizar Ruídos

Não será suavizado pois os dados parecem importantes no processo de classificação.

#Resolver Inconsistências

In [ ]:
print("Não tem atributos inconsistentes !")
print("Tentar remover instâncias duplicadas !")

print("Itens Existentes",df.count()[0])

#Remover Duplicados
df.drop_duplicates(inplace=True)
print("Itens Existentes após remoção:",df.count()[0])


Nessa Base de Dados não teve dados Duplicados.



In [ ]:
# Utilizado pelo Apriori
df_frequentes = df.copy()

#Transformar atributos categóricos nominais e ordinais em um vetor numérico, usando o OneHotEncoder)

In [ ]:
df.head()

In [ ]:
def aprender_labels(df, atributos):
  label_encoders = dict()
  
  for atributo in atributos:
    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(df[atributo])
    label_encoders[atributo] = label_encoder

  return label_encoders
  
# Transformar em numérico
def tranformar_em_numerico(df, label_encoders, atributos):
  for atributo in atributos:
    df[atributo] = label_encoders[atributo].transform(df[atributo])



In [ ]:
atributos_nominais = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Name", "Transported"]

for atributo in atributos_nominais:
  df[atributo] = df[atributo].astype(str)

label_encoders = aprender_labels(df, atributos_nominais)
tranformar_em_numerico(df, label_encoders, atributos_nominais)

df.head(10)

In [ ]:
def aprender_one_hot(df, atributos):
  hot_encoders = dict()
  
  for atributo in atributos:
    hot_encoder = preprocessing.OneHotEncoder()
    hot_encoder.fit(df[atributo].values.reshape(-1,1))
    hot_encoders[atributo] = hot_encoder

  return hot_encoders
  
# Transformar em numérico
def tranformar_onehot(df, hot_encoders, label_encoders, atributos):
  for atributo in atributos:
    X = hot_encoders[atributo].transform(df[atributo].values.reshape(-1,1)).toarray()
    #X[1]
    dfOneHot = pd.DataFrame(X, columns= [atributo + " " + label for label in label_encoders[atributo].classes_])
    df = pd.concat([df, dfOneHot], axis=1)
    df.drop([atributo], axis=1, inplace=True)
  return df

In [ ]:

# Fazer one hot enconding
atributos_onehot = ["HomePlanet", "CryoSleep", "Destination", "VIP"]
hot_encoders = aprender_one_hot(df, atributos_onehot)

# Fazer o One Hot Enconding
df = tranformar_onehot(df, hot_encoders, label_encoders, atributos_onehot)

df.head(10)

In [ ]:
print(df.columns)

Após o One Hot Encoding, será feita uma análise da correlação.

In [ ]:
# Impressao da matriz de correlacao dos dados
for atributo in df.columns:
  df[atributo] = df[atributo].astype(float)
sns.pairplot(pd.DataFrame(dict(df)))

In [ ]:
# análise de correlação

correlation = df.corr()

# plot da matriz de correlação
sns.set(rc = {'figure.figsize':(15,8)})
plot = sns.heatmap(correlation, annot = True, fmt=".1f", linewidths=.6, annot_kws = {'size':15})
plot

É possível observar que:
 - Existe uma correlação forte entre o Planeta de Partida e a Cabine.
 - Existem uma correlação moderada entre a Cabine de Hibernação e ser transportado para outra dimensão. Já analisado: se a pessoa foi para a Cabine de Hibernação, maior a chance de ser transportada para outra dimensão.
 -Existe uma correlação Moderada entre Planeta de Origem Europa e os gastos na praça de Alimentação (e em outras áreas da nave).

In [ ]:
atributos = ['HomePlanet Desconhecido', 'HomePlanet Earth', 'HomePlanet Europa', 'HomePlanet Mars']

for atributo in atributos:
  sns.set()
  gasto = df[df[atributo] == 1]
  gasto = gasto.FoodCourt

  plt.figure(figsize=(8, 6))
  plt.title("Gráfico de Distribuição - "+ atributo, fontsize=20)
  plt.xlabel("Gastos na Praça de Alimentação")
  plt.ylabel('Quantidade')
  
  plt.hist(gasto, alpha=0.5, label=atributo)

  plt.legend(loc='upper right')
  print("\n")

# Normalizando os valores de atributos numéricos.

In [ ]:
#Coloca Transported como ultimo atributo
def coloca_transported_ultimo(df):
  transported = list(df.Transported)

  df.drop("Transported", axis=1, inplace=True)
  df['Transported']= transported

In [ ]:
def normalizador(df):
  scaler = preprocessing.MinMaxScaler()
  scaler.fit(df)
  return scaler

In [ ]:
def normalizar(df, scaler):
  X = scaler.transform(df)
  df= pd.DataFrame(X, columns=df.columns)
  return df

In [ ]:
# Min Max Scaler

coloca_transported_ultimo(df)
aux = df.copy()

df.drop(["Transported"], axis=1, inplace=True)

scaler = normalizador(df)

df = normalizar(df, scaler)
df["Transported"] = aux["Transported"]

df.head()

Fazer uma verificação na correlação novamente.

In [ ]:
# análise de correlação
correlation = df.corr()

# plot da matriz de correlação
sns.set(rc = {'figure.figsize':(15,8)})
plot = sns.heatmap(correlation, annot = True, fmt=".1f", linewidths=.6, annot_kws = {'size':15})
plot

# Seleção de Atributos

In [ ]:
#Remover PassengerId
df.drop(["PassengerId", "Name"], axis=1, inplace=True)
df.head()

# Discretização dos Atributos Numéricos

In [ ]:
df_frequentes.drop(["PassengerId"], axis=1, inplace=True)
df_frequentes.head()

In [ ]:
print(df_frequentes.columns)

Discretizar Idade

In [ ]:
bins = [-1, 12, 18, 40, 60, df_frequentes.Age.max()+1]

df_frequentes["Age"] = pd.cut(df_frequentes["Age"].to_numpy(), 
                              bins=bins,
                              labels=["Criança", "Adolescente", "Adulto Jovem", "Adulto Velho", "Idoso"])

df_frequentes.head()

CryoSleep - Tornar mais Fácil de Entender

In [ ]:
rotulos = {False: 'Não Hibernou', True: 'Hibernou', "Desconhecido" : "Não sabe se Hibernou"}
df_frequentes['CryoSleep'] = df_frequentes['CryoSleep'].map(rotulos)
df_frequentes.head()

VIP - Tornar mais Fácil de Entender

In [ ]:
rotulos = {False: 'Não Vip', True: 'Vip', "Desconhecido": "Não sabe se é VIP"}
df_frequentes['VIP'] = df_frequentes['VIP'].map(rotulos)
df_frequentes.head()

Transported - Tornar mais Fácil de Entender

In [ ]:
rotulos = {False: 'Não Transportado', True: 'Transportado'}
df_frequentes['Transported'] = df_frequentes['Transported'].map(rotulos)
df_frequentes.head()

Discretizar Atributos - RoomService,	FoodCourt,	ShoppingMall,	Spa, VRDeck	

In [ ]:
atributos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

for atributo in atributos:
  bins = [-1, 500, 1000, df_frequentes[atributo].max()+1]

  df_frequentes[atributo] = pd.cut(df_frequentes[atributo].to_numpy(), 
                                bins=bins,
                                labels=["Gasto Pouco " + atributo, 
                                        "Gasto Médio " + atributo,
                                        "Gasto Muito " + atributo])

df_frequentes.head()

In [ ]:
df_frequentes.count()

# Encontrar Regras de Transações

Remover Espaços Antes de Depois dos atributos

In [ ]:
for atributo in df_frequentes.columns:
  df_frequentes[atributo] = df_frequentes[atributo].str.strip()

df_frequentes.head()

Transformar cada instância da base para o formato transacional

In [ ]:
transactions = []
for i in range(0, len(df_frequentes)):
    transactions.append([str(df_frequentes.values[i,j]) for j in range(0, len(df_frequentes.columns))])

transactions[0]

Codificar o One Hot para utilizar o apriori

In [ ]:
# Instantiate transaction encoder and identify unique items
encoder = TransactionEncoder().fit(transactions)

# One-hot encode transactions
onehot = encoder.transform(transactions)

# Convert one-hot encoded data to DataFrame
onehot = pd.DataFrame(onehot, columns = encoder.columns_)

# Print the one-hot encoded transaction dataset
onehot.head()

Gerar Itens Frequentes e Regras de Associação

In [ ]:
# Gerar Itens Frequentes
frequent_itemsets = apriori(onehot, min_support=0.07, use_colnames=True)

# Gerar Regras a Partir dos Items frequentes
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(1000)

Encontrar Regras Interessantes

In [ ]:
rules[ (rules['confidence'] >= 0.90) & (rules['support'] >= 0.75)  ]

Com mais de 95% de confiança e 75% de suporte:

  - Se uma pessoa gastou pouco em qualquer área então ela não é VIP

In [ ]:
rules[ (rules['confidence'] >= 0.95) & (rules['support'] >= 0.6)]

In [ ]:
rules[ (rules['lift'] >= 4)]

Analisando o lift, se a pessoa tiver partido de Europa então aumenta muita a chance dela ter gastado muito.


In [ ]:
rules[ (rules['confidence'] >= 0.95) & (rules['support'] >= 0.4)]

Com 95% de confiança e 40% de suporte, se uma pessoa foi transportada então ela gastou pouco com serviço de quarto.

# Classificação

Importações

In [ ]:
# Importações
#Modelos de aprendizado
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix


#Validacao cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Organizar Dados de treino

In [ ]:
# Base de Treino
df.head()

In [ ]:
# Dados de treino
train_x = df.values[:,:-1]
train_y = df.values[:, -1]

print(train_x.shape, train_y.shape)

Organizar Dados de teste

In [ ]:
# Ler dados de teste
df = pd.read_csv("test.csv")

# Tamanho da base de Dados
tamanho_do_df = len(df.index)
print("Tamanho da base de teste:", tamanho_do_df)

print("Algumas instâncias da base de teste:")
df.head()

In [ ]:
# Verificar se tem dados nulos
df.count()

In [ ]:
# Preencher dados nulos nos nominais
atributos = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Name"]

for atributo in atributos: 
  df[atributo] = imputers_nominais.transform(df[[atributo]])

df.count()

In [ ]:
# Preencher dados nulos Atributos Numéricos
atributos = ["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
for atributo in atributos: 
  df[atributo] = imputers_continuos[atributo].transform(df[[atributo]])

df.count()

In [ ]:
def realizar_label_encoder(atributo, label_encoders, novo_valor="Desconhecido"):
  df[atributo] = df[atributo].map(lambda valor: novo_valor if valor not in label_encoders[atributo].classes_ else valor)
  le_dict = dict(zip(label_encoders[atributo].classes_, label_encoders[atributo].transform(label_encoders[atributo].classes_)))
  if le_dict.get(novo_valor, "NaoAchou") == "NaoAchou":
    le_dict[novo_valor] = max(le_dict.values()) +1

  df[atributo] = df[atributo].map(lambda valor: le_dict[valor]) # Fazer Label Encoder)

In [ ]:
# Realizar Label Encoder na base teste
atributos_nominais = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Name"]

for atributo in atributos_nominais:
  realizar_label_encoder(atributo, label_encoders)


df.head()

In [ ]:
# Realizar One Hot Encoder na base teste
atributos_onehot = ["HomePlanet", "CryoSleep", "Destination", "VIP"]

df = tranformar_onehot(df, hot_encoders, label_encoders, atributos_onehot)

df.head()

In [ ]:
# Normalizar Base teste

df = normalizar(df, scaler)

df.head()

In [ ]:
# Fazer seleção de atributos base teste
df.drop(["PassengerId", "Name"], axis=1, inplace=True)
df.head()

In [ ]:
# Dados de teste
test_x = df.values[:,:]


print(test_x.shape)

Função para plotar a Matriz de Confusão

In [ ]:
def plotar_matriz_de_confusao(matriz_de_confusao, predicao):
  ax = sns.heatmap(matriz_de_confusao, annot=True, cmap='Blues')

  ax.set_title('Seaborn Confusion Matrix with labels\n\n')
  ax.set_xlabel('\nPredicted Values')
  ax.set_ylabel('Actual Values ')

Modelo 1 - Regressão Logística

In [ ]:
# Treinamento
modelo_regressao_logistica = LogisticRegression(C=0.2)
modelo_regressao_logistica.fit(train_x, train_y)

# KFolds
cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores_regressao_logistica_accuracy = cross_val_score(modelo_regressao_logistica, train_x, train_y, scoring='accuracy', cv=cv)

print("Regressão Logística")
print("Acurácia (Desvio Padrão)")
print('Acurácia Regressão Logística: %.3f (%.3f)' % (np.mean(scores_regressao_logistica_accuracy), np.std(scores_regressao_logistica_accuracy)))

In [ ]:
# Predicao
predicao_regresao_logistica = modelo_regressao_logistica.predict(test_x)

#print("Matriz de Confusão Regressão Logística")

#matriz_confusao_regressao_logistica = confusion_matrix(y_true = list(yTest), y_pred = list(predicao_regresao_logistica), labels=[1,2])
#plotar_matriz_de_confusao(matriz_confusao_regressao_logistica, predicao_regresao_logistica)

Modelo 2 - Árvore de Decisão

In [ ]:
# Treinamento

modelo_arvore_de_decisao = DecisionTreeClassifier(min_samples_split=30)
modelo_arvore_de_decisao.fit(train_x, train_y)

# KFolds
scores_arvore_de_decisao_accuracy = cross_val_score(modelo_arvore_de_decisao, train_x, train_y, scoring='accuracy', cv=cv)

print("Árvore de Decisão")
print("Acurácia (Desvio Padrão)")
print('Acurácia Árvore de Decisão: %.3f (%.3f)' % (np.mean(scores_arvore_de_decisao_accuracy), np.std(scores_arvore_de_decisao_accuracy)))

In [ ]:
# Predição
predicao_arvore_de_decisao = modelo_arvore_de_decisao.predict(test_x)

Modelo 3 - Random Forest

In [ ]:
# Treinamento
modelo_random_forest = RandomForestClassifier(n_estimators=110)
modelo_random_forest.fit(train_x, train_y)

#Kfolds

scores_random_forest_accuracy = cross_val_score(modelo_random_forest, train_x, train_y, scoring='accuracy', cv=cv)
print("Random Forest")
print("Acurácia (Desvio Padrão)")
print('Acurácia Random Forest: %.3f (%.3f)' % (np.mean(scores_random_forest_accuracy), np.std(scores_random_forest_accuracy)))

In [ ]:
# Predicao
predicao_random_forest = modelo_random_forest.predict(test_x)

Modelo 4 - Naive Bayes -> GaussianNB

In [ ]:
# Treinamento
modelo_naive_bayes = GaussianNB()
modelo_naive_bayes.fit(train_x, train_y)

#Kfolds

scores_naive_bayes_accuracy = cross_val_score(modelo_naive_bayes, train_x, train_y, scoring='accuracy', cv=cv)
print("Naive Bayes Gaussian")
print("Acurácia (Desvio Padrão)")
print('Acurácia Naive Bayes Gaussian: %.3f (%.3f)' % (np.mean(scores_naive_bayes_accuracy), np.std(scores_naive_bayes_accuracy)))

In [ ]:
# Predicao
predicao_naive_bayes = modelo_naive_bayes.predict(test_x)

Modelo 5 - KNN

In [ ]:
# Treinamento
modelo_knn = KNeighborsClassifier()
modelo_knn.fit(train_x, train_y)

#Kfolds

scores_knn_accuracy = cross_val_score(modelo_knn, train_x, train_y, scoring='accuracy', cv=cv)
print("KNN")
print("Acurácia (Desvio Padrão)")
print('Acurácia KNN: %.3f (%.3f)' % (np.mean(scores_knn_accuracy), np.std(scores_knn_accuracy)))


In [ ]:
#Predição
predicao_knn = modelo_knn.predict(test_x)

Modelo 6 - SVM

In [ ]:
# Treinamento

# Definir Parâmetros
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
modelo_svm = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
modelo_svm.fit(train_x, train_y)

#Kfolds

scores_svm_accuracy = cross_val_score(modelo_svm, train_x, train_y, scoring='accuracy', cv=cv)
print("SVM")
print("Acurácia (Desvio Padrão)")
print('Acurácia SVM: %.3f (%.3f)' % (np.mean(scores_svm_accuracy), np.std(scores_svm_accuracy)))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.711 total time=   4.8s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.726 total time=   3.1s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.710 total time=   4.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.723 total time=   4.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.733 total time=   3.1s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.711 total time=   4.4s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.723 total time=   3.8s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.708 total time=   3.8s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.727 total time=   3.3s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.731 total time=   3.6s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.710 total time=   6.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

In [ ]:
# Printar os Melhores Parâmetros e Estimadores
print(modelo_svm.best_params_)
print(modelo_svm.best_estimator_)

In [ ]:
# Predicao
predicao_svm = modelo_svm.predict(test_x)

# Modelo Extra - Rede Neural

Importações

In [ ]:
import os
import cv2
from keras.models import Sequential
from tensorflow.keras import layers, models
from keras.layers import  Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn import model_selection

In [ ]:
train_x.shape

In [ ]:
# Montar Modelo
FEATURES = train_x.shape[1]
model1 = models.Sequential()

model1.add(layers.InputLayer(input_shape=FEATURES))
model1.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))


model1.add(layers.Dense(256, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))



model1.add(layers.Dense(128, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))


model1.add(layers.Dense(64, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))

model1.add(layers.Dense(32, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))

"""model1.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))

model1.add(layers.Dropout(0.5))

model1.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))

model1.add(layers.Dropout(0.5))

model1.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))

model1.add(layers.Dropout(0.5))

model1.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))

model1.add(layers.Dropout(0.5))

model1.add(layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'))

model1.add(layers.Dropout(0.5))"""


model1.add(layers.Dense(2, activation='softmax', name='CamadaClassificacao'))
                       
model1.summary()

# Compilar
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Treinar
model1.fit(train_x, train_y, epochs=1000)

# Predição
#Obter Resultados
y_pred = model1.predict(test_x, verbose=1)
predicao_rede_neural = np.argmax(y_pred, axis=1)

print(list(predicao_rede_neural))

Gerar Arquivo de predição do Kaggle

In [ ]:
new_df = pd.read_csv("test.csv")

output = pd.DataFrame({'PassengerId': new_df.PassengerId, 'Survived': predicao_rede_neural})
output.to_csv('submission.csv', index=False)
print("Predictions Saved")